<a href="https://colab.research.google.com/github/Santosh-Gupta/SparseTorch/blob/master/Research2VecTraining8_4_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import math
import numpy as np
import random
import zipfile
import os
import tensorflow as tf
import pandas as pd
import pickle

!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from numpy import genfromtxt

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

vocabulary_size = 1666577 

# tf.logging.set_verbosity(tf.logging.ERROR)

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
# os.remove('adc.json')

In [0]:
#https://drive.google.com/open?id=1mdF0JfrzbOxeHD26JaUW8KEfIdMEHzQG

In [5]:
#link to hold dictionary folder https://drive.google.com/drive/u/0/folders/1q2D8xlzyQmUjggLStsfVEtgq3_GWxcKl
#in san gupta ml
dl_id = input("Enter Gdrive file ID for Title dictionary: ") # 9-10-18 1mdF0JfrzbOxeHD26JaUW8KEfIdMEHzQG

thefile = drive.CreateFile({'id': dl_id})
thefile.GetContentFile('titleDict.pickle')

with open('titleDict.pickle', 'rb') as handle:
    bookDictionary = pickle.load(handle)

Enter Gdrive file ID for Title dictionary: 1mdF0JfrzbOxeHD26JaUW8KEfIdMEHzQG


In [6]:
bookDictionary[5]

'One-way Loss Pattern Sample Metrics'

In [4]:
dl_id = input("Enter Gdrive file ID for Data ") # 9-10-18 1RHVwT1slwbhPlNTTF1JKS7agc4hQSm5c

myDownload = drive.CreateFile({'id': dl_id})
myDownload.GetContentFile('Data.npy')
my_data = np.load('Data.npy', allow_pickle=True)
print(my_data[0:15])

Enter Gdrive file ID for Data 1RHVwT1slwbhPlNTTF1JKS7agc4hQSm5c


ValueError: ignored

In [6]:
data_index = 0
epoch_index = 0
recEpoch_indexA = 0 #Used to help keep store of the total number of epoches with the models

def generate_batch(batch_size, inputCount): #batch size = number of labels
  #inputCount = number of inputs per label
    global data_index, epoch_index
    
    batch = np.ndarray(shape=(batch_size, inputCount), dtype=np.int32) 
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    n=0
    while n < batch_size:
      if len(    set(my_data[data_index, 1])   ) >= inputCount:
        labels[n,0] = my_data[data_index, 0]
        batch[n] = random.sample( set(my_data[data_index, 1]),  inputCount)
        n = n+1
        data_index = (data_index + 1) % len(my_data) #may have to do something like len my_data[:]
        if data_index == 0:
          epoch_index = epoch_index + 1
          print('Completed %d Epochs' % epoch_index)
      else:
        data_index = (data_index + 1) % len(my_data)
        if data_index == 0:
          epoch_index = epoch_index + 1
          print('Completed %d Epochs' % epoch_index)
    
    return batch, labels     
      
here, goes = generate_batch(20, 4) # to do next, insert %len(headernumber)
print('batch', here)
print('labels', goes)

batch [[ 510776  734702 1448326 1256014]
 [ 572983  431318  324439  168364]
 [ 951671  646875 1022675  910689]
 [1060750  914246  457861 1193128]
 [1437081  812990 1384755  300318]
 [ 145841  537201  877387  743717]
 [ 738525  381440 1406008  727615]
 [ 891973 1000807 1115942 1592057]
 [ 735748  260217  596114  576441]
 [ 881771  239877  863739  295417]
 [1255078 1395063 1651253  369818]
 [ 985816  769933 1171127   24147]
 [1369264 1135510  336068 1425951]
 [  33171 1067304 1060572  286200]
 [ 319694 1595297  543614  905572]
 [ 264237  381248  396562  145147]
 [ 391622  947622   35913 1325093]
 [1078309 1097516 1185471 1593042]
 [1011069 1499404 1474910  502310]
 [ 474993 1384644  410294  177826]]
labels [[ 1]
 [ 3]
 [ 4]
 [ 5]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [14]
 [15]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]
 [24]]


In [0]:
batch_size = 512 #2^8

embedding_size = 512 # 2^8 Dimension of the embedding vector.
num_inputs =4

num_sampled = 128 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(): #took out " , tf.device('/cpu:0')"
 

  train_dataset = tf.placeholder(tf.int32, shape=[batch_size, num_inputs ])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])


  epochCount = tf.get_variable( 'epochCount', initializer= 0) #to store epoch count to total # of epochs are known
  update_epoch = tf.assign(epochCount, epochCount + 1)

  embeddings = tf.get_variable( 'embeddings', 
    initializer= tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

  softmax_weights = tf.get_variable( 'softmax_weights',
    initializer= tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  
  softmax_biases = tf.get_variable('softmax_biases', 
    initializer= tf.zeros([vocabulary_size]),  trainable=False )

  embed = tf.nn.embedding_lookup(embeddings, train_dataset) #train data set is

  embed_reshaped = tf.reshape( embed, [batch_size*num_inputs, embedding_size] )
  
  segments= np.arange(batch_size).repeat(num_inputs)

  averaged_embeds = tf.segment_mean(embed_reshaped, segments, name=None)

  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=averaged_embeds,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss) #Original learning rate was 1.0
                                                                                                                              
  saver = tf.train.Saver()

In [0]:
def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [8]:
loadModel = input("Would you like to load a checkpoint? Type y or n: ") 

if loadModel == 'y':
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  
  zip_id = input("Enter Gdrive file ID for tensorflow models: ") 

  if not os.path.exists('checkpointsBook2VecCbowWindow1Downloaded'):
      os.makedirs('checkpointsBook2VecCbowWindow1Downloaded')

  # DOWNLOAD ZIP
  print ("Downloading zip file")
  myzip = drive.CreateFile({'id': zip_id})
  myzip.GetContentFile('model.zip')

  # UNZIP ZIP
  print ("Uncompressing zip file")
  zip_ref = zipfile.ZipFile('model.zip', 'r')
  zip_ref.extractall('checkpointsBook2VecCbowWindow1Downloaded/')
  zip_ref.close()

  print( os.getcwd() )
  print( os.listdir('./checkpointsBook2VecCbowWindow1Downloaded') )
  

Would you like to load a checkpoint? Type y or n: n


In [1]:
num_steps = 10000000

if 'loadModel' not in locals() and 'loadModel' not in globals():
  loadModel = 'n'

uploadModel = drive.CreateFile() #used to upload checkpoints when graph is run

with tf.Session(graph=graph) as session:
  
  if loadModel == 'y':
    saver.restore(session, './checkpointsBook2VecCbowWindow1Downloaded/Research2VecEmbedSize80.ckpt' )
  else:  
    tf.global_variables_initializer().run() #Don't initalize variables after a checkpoint has been restored
  
  print('Initialized')
  average_loss = 0
  saveIteration = 1
  for step in range(1, num_steps):
    
    batch_data, batch_labels = generate_batch(
      batch_size, num_inputs)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict) 

    average_loss += l
    if step % 8000 == 0:
      if step > 0:
        average_loss = average_loss / 8000
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
      
    if step % 50000 == 0:
      recEpoch_indexA =  epoch_index - recEpoch_indexA #how much did the epoch_index since it was last checked
      for nE in range(0, recEpoch_indexA ):
        session.run(update_epoch) #session run calls tend to be huge bottlenecks, keep in mind while determining the frequency
      recEpoch_indexA = epoch_index
      print('recEpoch_indexA is', recEpoch_indexA)
      print( 'epochCount.eval() is ', epochCount.eval() )
      print('epoch_index is ' , epoch_index)
      
      save_path = saver.save(session, "checkpointsBook2Vec5Inputs/Research2VecEmbedSize80.ckpt") #Save checkpoint
      
    #   auth.authenticate_user()
    #   gauth = GoogleAuth() #Gdrive authenticion code placed here since it expires after some time
    #   gauth.credentials = GoogleCredentials.get_application_default()
    #   drive = GoogleDrive(gauth) 
    #   uploadModel = drive.CreateFile() #Need to also create drive object with updated authenticion
      
    #   chptName = 'Research2VecEmbedSize80'+str(saveIteration)
    #   zipfolder(chptName, 'checkpointsBook2Vec5Inputs')
    #   uploadModel.SetContentFile(chptName+".zip")
    #   uploadModel.Upload()
      
    #   print("Checkpoint uploaded to Google Drive")
    #   saveIteration += 1
    #   os.remove(chptName+".zip") #Remove checkpoint zip file after upload

                


NameError: ignored